In [43]:
from qutip import *
import numpy as np
import math
from scipy.linalg import expm
from IPython.display import Latex

All Hamiltonian outputs are in units of frequency (Hz)

In [44]:
def z_hamiltonian(m, n, B):
    e = 1.6e-19
    h = 6.625e-34

    ge = 1.9987213544
    gn = -2.26
    mu_B = 5.7962605e-5 #eV/T
    mu_n = 3.156739525e-8 #eV/T
    H_z = np.zeros([2**(m+n), 2**(m+n)])
    for j in range(m+n):
        if j<m:
            C = 0.5*gn*mu_n
        else:
            C = 0.5*ge*mu_B
            
        Dx = B[0]*C*np.kron(np.eye(2**j), sigmax())
        Dy = B[1]*C*np.kron(np.eye(2**j), sigmay())
        Dz = B[2]*C*np.kron(np.eye(2**j), sigmaz())
        
        H_z = H_z+np.kron(Dx, np.eye(2**(m+n-j-1)))+np.kron(Dy, np.eye(2**(m+n-j-1)))+np.kron(Dz, np.eye(2**(m+n-j-1)))
    return H_z*e/h

In [45]:
# A is in MHz
def hf_hamiltonian(m, n, A):
    H_hf = np.zeros(2**(m+n))
    for j in range(m):
        Dx = np.kron(np.eye(2**j), sigmax())
        Dy = np.kron(np.eye(2**j), sigmay())
        Dz = np.kron(np.eye(2**j), sigmaz())
        
        for k in range(n):
            Dx_temp = np.kron(Dx, np.eye(2**(m+k-j-1)));
            Dx_temp = A[j][k]*np.kron(Dx_temp, sigmax());
            Dy_temp = np.kron(Dy, np.eye(2**(m+k-j-1)));
            Dy_temp = A[j][k]*np.kron(Dy_temp, sigmay());
            Dz_temp = np.kron(Dz, np.eye(2**(m+k-j-1)));
            Dz_temp = A[j][k]*np.kron(Dz_temp, sigmaz());
            H_hf = (H_hf+np.kron(Dx_temp, np.eye(2**(n-k-1)))+
                np.kron(Dy_temp, np.eye(2**(n-k-1)))+np.kron(Dz_temp, np.eye(2**(n-k-1))))

    return H_hf*1e6/4 #Hz

In [46]:
## tau[0] is tunneling time for up electron
## tau[1] is tunneling time for down electron

def outOp(m, evecs, tau, emptyOpt):
    l = (2**m)*3
    L_out = []
    for i in range(2**m):
        for k in range(l):
            initIndex = 2**(m+1)+i
            finIndex = int(3*np.ceil((k+1)/3)-1)
            L_temp = np.abs(evecs[initIndex][k, 0])*np.sqrt(1/tau[0])*Qobj([((fock(l, finIndex).dag()).full()).dot(val)[0] for val in evecs])*fock(l, initIndex).dag()
        
            if L_temp.norm() != 0:
                L_out.append(L_temp)
        
            if emptyOpt == 1:
                initIndex = i
                L_temp = np.abs(evecs[initIndex][k, 0])*np.sqrt(1/tau[1])*Qobj([((fock(l, finIndex).dag()).full()).dot(val)[0] for val in evecs])*fock(l, initIndex).dag()
            
                if L_temp.norm() != 0:
                    L_out.append(L_temp)
    return L_out